In [1]:
# pip install -r requirements.txt
import numpy as np
import pandas as pd
import os

/Users/charlotteverity/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
streaming_pre = './streaming_history_files'
streaming_paths = os.listdir(streaming_pre)
audio_paths = [x for x in streaming_paths if "Audio" in x]
audio_2025_paths = [x for x in audio_paths if "2025" in x]
audio_2025_paths

['Streaming_History_Audio_2025_13.json',
 'Streaming_History_Audio_2024-2025_12.json',
 'Streaming_History_Audio_2025_14.json']

## Helper Funcs

In [3]:
# Takes in list of data file paths of streaming data jsons and date limits
def load_streaming_files(paths, min_date = None, max_date = None):
  streaming_data = pd.DataFrame()
  for p in sorted(paths):
    df_temp = pd.read_json(streaming_pre+"/"+p)
    df_temp["stream_date"] = pd.to_datetime(df_temp["ts"]).dt.date
    streaming_data = pd.concat([streaming_data, df_temp])
  if(min_date is not None):
    streaming_data = streaming_data[(streaming_data["stream_date"]>=pd.to_datetime(min_date).date())]
  if(max_date is not None):
    streaming_data = streaming_data[(streaming_data["stream_date"]<=pd.to_datetime(max_date).date())]
  streaming_data = streaming_data[~streaming_data["master_metadata_track_name"].isna()]
  return streaming_data

# Keep necessary, non-PI columns
def clean_streaming_data(streaming_data):
  streaming_data = streaming_data[['platform', 'ms_played', 'master_metadata_track_name', 'master_metadata_album_artist_name', 
                               'master_metadata_album_album_name', 'spotify_track_uri', 'reason_start', 'reason_end', 
                               'skipped', 'stream_date']].copy()
  streaming_data["seconds"] = streaming_data["ms_played"]/1000
  streaming_data["minutes"] = streaming_data["seconds"]/60
  return streaming_data

def get_summary_stats(streaming_data):
  minutes_played = streaming_data["minutes"].sum()
  songs_played = streaming_data["spotify_track_uri"].count()
  unique_songs_played = streaming_data.groupby(["master_metadata_track_name", "master_metadata_album_artist_name"]).ngroups
  artists_played = streaming_data["master_metadata_album_artist_name"].nunique()
  print("Total minutes played: "+str(minutes_played))
  print("Total songs played: "+str(songs_played))
  print("Unique songs played: "+str(unique_songs_played))
  print("Unique artists played: "+str(artists_played))

def get_most_played_songs(streaming_data):
  most_played_songs = streaming_data.groupby(["master_metadata_track_name", "master_metadata_album_artist_name"])["master_metadata_track_name"]\
                                    .count().reset_index(name="plays").sort_values("plays", ascending=False)
  print("Most played songs: ")
  display(most_played_songs.head(5))
  return most_played_songs

def get_longest_played_songs(streaming_data):
  longest_played_songs = streaming_data.groupby(["master_metadata_track_name", "master_metadata_album_artist_name"])["minutes"]\
                         .sum().reset_index(name="minutes").sort_values("minutes", ascending=False)
  print("Longest played songs: ")
  display(longest_played_songs.head(5))
  return longest_played_songs

def get_most_played_artists(streaming_data):
  most_played_artists = streaming_data.groupby(["master_metadata_album_artist_name"])["master_metadata_track_name"]\
                         .count().reset_index(name="plays").sort_values("plays", ascending=False)
  print("Most played artists: ")
  display(most_played_artists.head(5))
  return most_played_artists

def get_longest_played_artists(streaming_data):
  longest_played_artists = streaming_data.groupby(["master_metadata_album_artist_name"])["minutes"]\
                          .sum().reset_index(name="minutes").sort_values("minutes", ascending=False)
  print("Longest played artists: ")
  display(longest_played_artists.head(5))
  return longest_played_artists

  


## 2025 Streams

In [4]:
audio_2025_df = load_streaming_files(audio_2025_paths, "2025-01-01", "2025-12-31")
audio_2025_df = clean_streaming_data(audio_2025_df)
print(audio_2025_df["stream_date"].min())
print(audio_2025_df["stream_date"].max())
print(audio_2025_df.shape)

2025-01-01
2025-10-09
(22921, 12)


In [5]:
get_summary_stats(audio_2025_df)

Total minutes played: 38921.6226
Total songs played: 22921
Unique songs played: 5178
Unique artists played: 1799


In [6]:
most_played_songs_2025 = get_most_played_songs(audio_2025_df)
longest_played_songs_2025 = get_longest_played_songs(audio_2025_df)
most_played_artists_2025 = get_most_played_artists(audio_2025_df)
longest_played_artists_2025 = get_longest_played_artists(audio_2025_df)

Most played songs: 


,master_metadata_track_name,master_metadata_album_artist_name,plays
2858,Nettles,Ethel Cain,104
2108,Illegal,PinkPantheress,89
4322,Tonight,PinkPantheress,86
4152,The Edge,Sydney Ross Mitchell,81
5050,party 4 u,Charli xcx,79


Longest played songs: 


,master_metadata_track_name,master_metadata_album_artist_name,minutes
2858,Nettles,Ethel Cain,698.383150
1528,Fuck Me Eyes,Ethel Cain,351.318450
4152,The Edge,Sydney Ross Mitchell,251.258967
5050,party 4 u,Charli xcx,245.392267
3974,Sunshine Baby,The Japanese House,204.949717


Most played artists: 


,master_metadata_album_artist_name,plays
303,Charli xcx,728
1720,Zara Larsson,641
516,Ethel Cain,586
334,Clairo,584
1247,PinkPantheress,544


Longest played artists: 


,master_metadata_album_artist_name,minutes
516,Ethel Cain,2103.026683
303,Charli xcx,1342.694650
334,Clairo,1245.126350
1720,Zara Larsson,1208.443650
1543,The Marías,1066.552733


## All-Time Streaming

In [ ]:
audio_all_df = load_streaming_files(audio_paths)
audio_2025_df = clean_streaming_data(audio_2025_df)
print(audio_2025_df["stream_date"].min())
print(audio_2025_df["stream_date"].max())
print(audio_2025_df.shape)